# Download Shapefile

In [ ]:
%%html 
<style>
.tooltip {
    position: relative;
    display: inline-block;
    border-bottom: 1px dotted black;
}

.tooltip .tooltiptext {
    visibility: hidden;
    width: 220px;
    background-color: white;
    color: black;
    text-align: center;
    border-radius: 6px;
    padding: 5px 0;
    border: 1px solid black;
    /* Position the tooltip */
    position: absolute;
    z-index: 1;
}

.tooltip:hover .tooltiptext {
    visibility: visible;
}
</style>
<div class="tooltip">
  <span class="tooltiptext">
<img src="https://upload.wikimedia.org/wikipedia/commons/3/38/Simple_vector_map.svg" style="max-width:100%;">
</span>
</div>


In [2]:
from IPython.display import  HTML

def load_d3_in_cell_output():
  display(HTML("<script src='https://d3js.org/d3.v6.min.js'></script>"))
get_ipython().events.register('pre_run_cell', load_d3_in_cell_output)

In [3]:
%%html
<script type="text/javascript">   

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { console.log(map) });
   
</script>

Where to get the csv of our data?? 

In [4]:
%%html
<div id="gohere4"></div>
<script type="text/javascript">   

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://gist.githubusercontent.com/dudaspm/57951a2fa648fa23b4197768bbac2907/raw/ef63f7bf27d7dbcfffafe5deebe3b28f2ce5ddf0/data_EPA_county.csv")
          .then(function(data) { 
            const chooseState = 36
            const epa = data.map(d=> ({fips:d.fips,pct_pub_transport:+d.pct_pub_transport}) )
            const PAEPAData = epa.filter(d=> d.fips.substring(0, 2) == chooseState) 
            const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)

            const svg = d3.select("div#gohere4").append("svg")
                .attr("width", width)
                .attr("height", height)
            var EPAKeys = {}
            PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 
            console.log(EPAKeys)
            // construct the element
            svg.selectAll("path").append('path')
              .data(ny.features)
              .join("path")
              .attr('d', pathGeneratorNY)
              .attr("fill", d=>palette(color(EPAKeys[d.properties.GEOID])))
              .attr('stroke', '#999999')
              .attr('stroke-width', '2')
                .append("title")
                .text(d => "the percent public transportation is: \n" + EPAKeys[d.properties.GEOID]);


          })
       });
   
</script>

### Adding Zoom Feature

In [28]:
%%html
<div id="gohere5"></div>
<script type="text/javascript">   

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://raw.githubusercontent.com/ArwaHarar/DS440/main/coords_density_df.csv")
          .then(function(data) { 
            console.log(data)
            const chooseState = 36
            const tweets = data.map(d=> ({Coords:d.Coords.replace("[","").replace("]","").split(","),Timestamp:d.Timestamp,Tweet:d.Tweet,Density:+d.Density}) )
            console.log(tweets)
            //const PAEPAData = tweets.filter(d=> d.fips.substring(0, 2) == chooseState) 
            //const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)

            const svg = d3.select("div#gohere5").append("svg")
                .attr("width", width)
                .attr("height", height)
            var EPAKeys = {}
            //PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 



            ////////////////////////////////////////
            // ZOOOOOOOOOOOOOOOOOOOOOOOOMING
            svg.call(d3.zoom()
                .extent([[0, 0], [width, height]])
                .scaleExtent([1, 15])
                .on("zoom", zoomed));

            function zoomed({transform}) {
              g.attr("transform", transform);
            }
            const g = svg.append("g");
            ////////////////////////////////////////
            // construct the element
            g.selectAll("path").append('path')
              .data(ny.features)
              .join("path")
              .attr('d', pathGeneratorNY)
              //.attr("fill", d=>palette(color(EPAKeys[d.properties.GEOID])))
              .style("fill","white")
              .attr('stroke', '#999999')
              .attr('stroke-width', '2')

            g.selectAll("circle").append('circle')
              .data(tweets)
              .join("circle")
              //.attr("cx",d=> console.log(projectionNY([d.Coords[0],d.Coords[1]])[0])))
              .attr("cx",d=> projectionNY([d.Coords[0],d.Coords[1]])[0])
              .attr("cy",d=> projectionNY([d.Coords[0],d.Coords[1]])[1])
              .attr("r",1)
              .style("fill","green")
              .style("opacity",.05)
                .append("title")
                .text(d => d.Density);
              

          })
       });
   
</script>

In [ ]:
"%%html
<div id="gohere6"></div>
<script type="module"> 
    import * as d3 from "https://cdn.skypack.dev/d3@7"; 
    import {tile} from "https://cdn.skypack.dev/d3-tile@1";

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://gist.githubusercontent.com/dudaspm/57951a2fa648fa23b4197768bbac2907/raw/ef63f7bf27d7dbcfffafe5deebe3b28f2ce5ddf0/data_EPA_county.csv")
          .then(function(data) { 
            const chooseState = 36
            const epa = data.map(d=> ({fips:d.fips,pct_pub_transport:+d.pct_pub_transport}) )
            const PAEPAData = epa.filter(d=> d.fips.substring(0, 2) == chooseState) 
            const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)
            
            const svg = d3.select("div#gohere6").append("svg")
                .attr("width", width)
                .attr("height", height)

            var EPAKeys = {}
            PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 
            console.log(EPAKeys)
            // construct the element

            const tiles = tile()
                .extent([[0, 0], [width, height]])
                .tileSize(512)
                .clampX(false);
            const zoom = d3.zoom()
                .scaleExtent([1 << 12, 1 << 20])
                .extent([[0, 0], [width, height]])
                .on("zoom", ({transform}) => zoomed(transform));


            console.log("44") 
            let image = svg.append("g")
                .attr("pointer-events", "none")
              .selectAll("image");
            console.log("48")

    
            console.log("55")

            const path = svg.append("path")
                .attr("pointer-events", "none")
                .attr("fill", "none")
                .attr("stroke", "red")
                .attr("stroke-linecap", "round")
                .attr("stroke-linejoin", "round");

            const initialCenter = [-73.935242,40.730610]
            const initialScale = 1 << 12
            console.log((projectionNY(initialCenter)))
            svg
                .call(zoom)
                .call(zoom.transform, d3.zoomIdentity
                    .translate(width / 2, height / 2)
                    .scale(-initialScale)
                    .translate(...projectionNY(initialCenter))
                    .scale(-1));


            function zoomed(transform) {
              /*const tiles = tile(transform);
              console.log(tiles())

              image = image.data(tiles, d => d).join("image")
                  .attr("xlink:href", d => url(...d3.tileWrap(d)))
                  .attr("x", ([x]) => (x + tiles.translate[0]) * tiles.scale)
                  .attr("y", ([, y]) => (y + tiles.translate[1]) * tiles.scale)
                  .attr("width", tiles.scale)
                  .attr("height", tiles.scale);*/

              console.log("asdf")
              projectionNY
                  .scale(transform.k / (2 * Math.PI))
                  .translate([transform.x, transform.y]);

              path.attr("d", pathGeneratorNY(ny));
            }
            function url(x,y,z) {
              return "http://" + ["a", "b", "c"][Math.random() * 3 | 0] + ".tile.openstreetmap.org/" + z + "/" + x + "/" + y + ".png";
            }

          })
       });
   
</script>